selenium, bs4를 설치해 브라우저를 원격 조정하자.

컴퓨터가 최소 8GB 이상, i5 이상의 CPU, Windows 환경을 추천한다.

Update: 2020/08/11 - remove previous comments 부분 해결하기. click more 부분 처리(bug fix 필요)

In [ ]:
!pip install selenium

In [ ]:
!pip install bs4

Firefox를 설치하자.

다음으로 geckodriver를 설치해야 한다.

https://github.com/mozilla/geckodriver/releases 에서 본인에게 맞는 OS를 골라 설치하자.

압축을 풀고 C:/Windows 폴더에 geckodriver.exe를 복사한다. 이때 관리자 권한이 필요하다. 리눅스나 mac사용자는 /usr/bin 으로 geckodriver를 옮기면 된다.

예를 들어, Linux 사용자는 터미널에서
```
    $ wget https://github.com/mozilla/geckodriver/releases/download/v0.27.0/geckodriver-v0.27.0-linux64.tar.gz
    $ sudo sh -c 'tar -x geckodriver -zf geckodriver-v0.27.0-linux64.tar.gz -O > /usr/bin/geckodriver'
    $ sudo chmod +x /usr/bin/geckodriver
    $ rm geckodriver-v0.73.0-linux64.tar.gz
```



In [1]:
from selenium import webdriver #웹드라이브
from selenium.webdriver.firefox.options import Options #Firefox options
from selenium.webdriver.common.keys import Keys #key emulation
from bs4 import BeautifulSoup as BS

In [2]:
import time

In [3]:
'''
Factory - Firefox
'''
def fire_fox_browser(headless=False,image_allow=True):
    options=Options()
    profile=webdriver.FirefoxProfile()
    profile.set_preference('permissions.default.image',1 if image_allow else 2)
    options.headless=headless
    driver=webdriver.Firefox(firefox_profile=profile,options=options)
    return driver

In [6]:
youtube_url="https://www.youtube.com/watch?v=9kU6jb8XXpw" #펭TV 남극 동물의 한국식 여름 휴가, 코멘트 많은
youtube_url_no_comment="https://www.youtube.com/watch?v=S2qJAO03SsU" #코멘트 없는
youtube_url_few_comment="https://www.youtube.com/watch?v=-5WjyN53Zw4" #코멘트 적은

In [4]:
browser=fire_fox_browser(image_allow=True)

In [8]:
browser.quit()

In [56]:
from selenium import webdriver #webdriver
from selenium.webdriver.firefox.options import Options #Firefox options
from selenium.webdriver.common.keys import Keys #key emulation
from bs4 import BeautifulSoup as BS
# utility
import time

#' Firefox factor, inherited from webdriver.Firefox
class FactoryFirefoxDriver(webdriver.Firefox):
    def __init__(self,headless=False,image_allow=True):
        options=Options()
        profile=webdriver.FirefoxProfile()
        profile.set_preference('permissions.default.image',1 if image_allow else 2)
        options.headless=headless #headless mode on?
        super().__init__(firefox_profile=profile,options=options)
class YouTubeCommentBrowser(FactoryFirefoxDriver):
    def __init__(self,headless=False,image_allow=True):
        super().__init__(headless,image_allow)
    def press_down(self,h,tb=1):
        h.send_keys(Keys.END)
        time.sleep(tb)
    def scroll_down(self,tb=1):
        initial_height = self.execute_script("return window.scrollY")
        html=self.find_element_by_tag_name('html')
        self.press_down(html,tb)
        time.sleep(tb)
        #self.press_down(html)
        last_height = self.execute_script("return window.scrollY")
        if initial_height == last_height:
            return False
        while True:
            time.sleep(tb)
            new_height=self.execute_script("return window.scrollY")
            if new_height==last_height:
                break
            last_height=new_height
        return True
    def remove_element_by_id(self,s,tb=0.5):
        script="document.getElementById('{}').remove();".format(s)
        self.execute_script(script)
        time.sleep(tb)
    def remove_comments(self,tb=0.5):
        script='''
        document.querySelectorAll("#replies").forEach(function(item,index){item.remove();});
        var reviews=document.querySelectorAll('#body');
        reviews.forEach(function(item,index){
        if(index<(reviews.length-1)) item.remove();
        })
        '''
        self.execute_script(script)
        time.sleep(tb)
    def reply_more(self,tb=1):
        code_view='//yt-formatted-string[starts-with(.,"View") and contains(.,"reply")]'
        code_show_more_replies='//yt-formatted-string[starts-with(.,"View") and contains(.,"replies")]'
        eles=self.find_elements_by_xpath(code_view)
        for ele in eles:
            try:
                if ele.text is not "":
                    ele.click()
                    time.sleep(0.1)
            except:
                pass
        eles=self.find_elements_by_xpath(code_show_more_replies)
        for ele in eles:
            try:
                if ele.text is not "":
                    ele.click()
                    time.sleep(0.1)
            except:
                pass
        time.sleep(tb)
    def text_more(self,tb=1):
        code_read_more='//span[starts-with(.,"Read more")]'
        eles=self.find_elements_by_xpath(code_read_more)
        for ele in eles:
            try:
                if ele.text is not "":
                    ele.click()
                    time.sleep(0.1)
            except:
                pass
        time.sleep(tb)
    def visit(self,url,remove_ad=True,remove_player=False,tb=5):
        self.get(url)
        time.sleep(tb)
        if remove_ad:
            self.remove_element_by_id('related')
            time.sleep(0.1)
        if remove_player:
            self.remove_element_by_id('movie_player')
            time.sleep(0.1)
    def get_data(self):
        page=self.page_source
        soup=BS(page,'html.parser')
        return [ele.text for ele in soup.select('#content-text')]
    def collect_comments(self,verbose=False,remove_previous=True):
        total_comment=[] # //TODO - bs4
        if verbose:
            print("Start.")
        while True:
            if verbose:
                print("..Scroll down")
            if self.scroll_down(3) is False:
                break
            if verbose:
                print("..Call more replies, read more")
            for i in range(3):
                self.reply_more()
            self.text_more()
            if verbose:
                print("..Collect data")
                
            if remove_previous:
                if verbose:
                    print("..Remove previous comments in the browser")
                total_comment+=self.get_data()
                self.remove_comments()
        if remove_previous is False:
            total_comment=self.get_data()
        if verbose:
            print("Done.")
        return total_comment


In [57]:
b=YouTubeCommentBrowser(headless=False,image_allow=True)

In [58]:
b.visit(youtube_url_few_comment)

In [16]:
b.scroll_down()

True

In [55]:
b.quit()

In [61]:
b.text_more()

In [62]:
b.reply_more()

```
#content-text
```

In [73]:
eles=b.find_elements_by_id("comment")

In [74]:
eles[2].text

'지점토\n8 months ago\n핑퐁이라는 광운훌리 댓글마다보이네 ㅋㅋㅋㅋㅋㅋㅋㅋㅌㅌ니가 그러는게 학교이미지 까내리는거야^^\n41\nREPLY'

In [75]:
ele=eles[2]

In [76]:
ele.find_element_by_id('author-text').text

'지점토'

In [77]:
ele.find_element_by_id('content-text').text

'핑퐁이라는 광운훌리 댓글마다보이네 ㅋㅋㅋㅋㅋㅋㅋㅋㅌㅌ니가 그러는게 학교이미지 까내리는거야^^'

In [72]:
ele.id

'bd3086cc-7626-42af-9878-ec8325849b57'

In [82]:
replies=ele.find_element_by_xpath("following-sibling::*[1]")

In [83]:
replies.text

'Hide 13 replies\n시이잉\n8 months ago\n단국훌리도 만만치 않긔윤ㅋㅋㅋㅋ\n5\nREPLY\n지점토\n8 months ago\n시이잉 난 충남훌리인데?\nREPLY\n시이잉\n7 months ago\n@지점토 그건 별로 안궁금하궁ㅎㅎ!\nREPLY\n지점토\n7 months ago\n시이잉 넌 팡운이냐?ㅋㅋㅋㅋ딱 그정도네\n6\nREPLY\n시이잉\n7 months ago\n왜그랭 도스트야 친하게 지내장~\nREPLY\n남궁찬\n6 months ago\n@지점토 웩\nREPLY\n지점토\n6 months ago\n남궁찬 충경외시다 외워라\nREPLY\nSAM SUNG\n5 months ago\n@지점토 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㄹㅇ 훌리값하노 ㅇㅂ준다ㅇㄱ\nREPLY\n지점토\n5 months ago\n이도류 고맙다 ㅇㄱㅇ\nREPLY\n지웅\n2 months ago\n충남대는 낄때 끼자... 지방대 특성화고애들이 다니는 학교를 무슨..  평백도 80이면 가는학교구만ㅋㅋ\nREPLY\nShow more replies'

In [ ]:
class Comment:
    def __init__(self):
        self.id=-1
        self.parent=None
        self.replies=[]
class YouTubeCommentInit:
    def __init__(self):
        pass
class YouTubeCommentFactory(YouTubeCommentInit):
    def __init__(self):
        super().__init__()